In [1]:
# Dependencies and libraries #

import os
import requests
import json
import pandas as pd
import datetime
from datetime import date, timedelta
import hvplot
import alpaca_trade_api as tradeapi
from finta import TA
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

%matplotlib inline

In [2]:
# Set Alpaca API key and secret by calling the os.getenv function and referencing the environment variable names
# Set each environment variable to a notebook variable of the same name
alpaca_api_key = "PKT126TVACUEXXFT8Z85"
alpaca_secret_key = "WwlEmn0WUNYY0iJL7ltRQypkzqNfojvcnFtScBIy"

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [3]:
# Create your Alpaca API REST object by calling Alpaca's tradeapi.REST` function
# Set the parameters to your alpaca_api_key, alpaca_secret_key and api_version="v2" 
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")


In [4]:
ticker = "TSLA"

In [5]:
# Dates
start_date = pd.Timestamp("2019-10-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-10", tz="America/New_York").isoformat()

In [6]:
# Timeframe 
timeframe = "1Day"

In [7]:
# Price information #
df = alpaca.get_bars(
    ticker,
    timeframe,
    start = start_date,
    end = end_date,
).df

# Preview the resulting `portfolio_prices_df` DataFrame. 
df

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2019-10-01 04:00:00+00:00,241.50,245.9500,239.1300,244.69,6209990,101685,242.666097
2019-10-02 04:00:00+00:00,243.29,244.6500,239.4300,243.13,6256548,94931,241.362570
2019-10-03 04:00:00+00:00,232.04,234.4800,224.2800,233.03,15137764,230597,229.266463
2019-10-04 04:00:00+00:00,231.82,234.7800,228.0700,231.43,8025180,106994,230.830586
2019-10-07 04:00:00+00:00,230.00,238.5600,228.5500,237.72,8086958,97332,234.987850
...,...,...,...,...,...,...,...
2022-10-04 04:00:00+00:00,250.50,257.5000,242.0100,249.44,110028529,1313286,249.704849
2022-10-05 04:00:00+00:00,245.00,246.6697,233.2700,240.81,87581871,1172525,238.492134
2022-10-06 04:00:00+00:00,239.52,244.5800,235.3500,238.13,69296298,827199,239.355660


In [8]:
#create OHLCV df; drop 'trade_count' and 'vwap'columns
df = df.drop(['trade_count', 'vwap'], axis=1)

# Use the pct_change function to generate the returns from "close"
df["actual_returns"] = df["close"].pct_change()

# Drop all NaN values from the DataFrame
df = df.dropna()

# Generate the Input Features, X
# Create additional technical indicators
df['sma_slow'] = TA.SMA(df, 100)
df['sma_fast'] = TA.SMA(df, 4)
df["ssma"] = TA.SSMA(df)
df["ema"] = TA.EMA(df, 50)
df["dema"] = TA.DEMA(df)
df["tema"] = TA.TEMA(df)
df["trima"] = TA.TRIMA(df)
df["trix"] = TA.TRIX(df)
df["vama"] = TA.VAMA(df)
df["kama"] = TA.KAMA(df)
df["zlema"] = TA.ZLEMA(df)
df["wma"] = TA.WMA(df)

df = df.dropna()

df

,open,high,low,close,volume,actual_returns,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,,,,,,,
2020-02-25 05:00:00+00:00,849.00,856.6000,787.0000,799.91,17320981,-0.040634,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556
2020-02-26 05:00:00+00:00,782.45,813.3100,776.1100,778.80,14163418,-0.026390,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333
2020-02-27 05:00:00+00:00,730.21,739.7700,669.0000,679.00,24277832,-0.128146,456.8824,772.8750,774.704328,644.116213,785.502081,741.621826,691.737315,1.830608,837.616079,818.665230,855.457409,807.005333
2020-02-28 05:00:00+00:00,625.81,690.5200,611.5200,667.99,24566968,-0.016215,461.2480,731.4250,762.847116,645.067891,742.013136,692.104305,704.958056,1.744067,813.707288,806.553958,833.670395,774.653111
2020-03-02 05:00:00+00:00,711.53,743.6900,686.6700,743.62,20201438,0.113220,466.3070,717.3525,760.710760,648.993922,738.446086,699.553803,716.517191,1.654336,783.969571,805.307392,825.239568,760.361111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-04 04:00:00+00:00,250.50,257.5000,242.0100,249.44,110028529,0.029043,631.3831,256.3250,298.866032,461.343013,249.105834,254.201074,341.503056,-2.395481,266.493956,270.730688,236.891373,264.183556
2022-10-05 04:00:00+00:00,245.00,246.6697,233.2700,240.81,87581871,-0.034597,626.5112,249.4750,292.415362,452.694659,242.862784,246.528420,328.244568,-2.406676,262.542500,265.216749,232.125345,258.212667
2022-10-06 04:00:00+00:00,239.52,244.5800,235.3500,238.13,69296298,-0.011129,621.1966,242.6950,286.383655,444.280359,237.820321,240.814765,316.680463,-2.415128,258.923003,260.795585,227.340505,252.767556


In [9]:
# Assign a copy of the technical variable columns to a new DataFrame called `X` and shift values.
# The shifted 'X' values will align a prior day's X values with the next day's 'y'/returns/trading signal,
# to train the predictive model. 
X = df[['sma_slow', 'sma_fast', 'ssma', 'ema', 'dema', 'tema', 'trima', 'trix', 'vama', 'kama', 'zlema', 'wma']].shift().dropna().copy()

# Initialize a `signal` column
df['signal'] = 0.0

# signal values will be based on the daily returns: positive returns yield '1', negative returns '-1'
df.loc[(df['actual_returns'] >= 0), 'signal'] = 1
df.loc[(df['actual_returns'] < 0), 'signal'] = -1

# Copy the 'signal' column to a new Series called `y`.
y = df['signal']

display(df.head(2))
display(X.head(2))
display(X.tail(2))
display(y.head(2))

,open,high,low,close,volume,actual_returns,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma,signal
timestamp,,,,,,,,,,,,,,,,,,,
2020-02-25 05:00:00+00:00,849.00,856.60,787.00,799.91,17320981,-0.040634,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556,-1.0
2020-02-26 05:00:00+00:00,782.45,813.31,776.11,778.80,14163418,-0.026390,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333,-1.0


,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,
2020-02-26 05:00:00+00:00,447.0660,858.5275,787.650889,637.006829,866.700630,846.870644,662.158488,1.932171,860.113822,824.581514,882.381335,852.645556
2020-02-27 05:00:00+00:00,452.4227,828.3750,786.667450,642.666895,840.604053,812.379254,677.211173,1.895748,857.957744,824.336295,876.979214,839.933333


,sma_slow,sma_fast,ssma,ema,dema,tema,trima,trix,vama,kama,zlema,wma
timestamp,,,,,,,,,,,,
2022-10-07 04:00:00+00:00,621.1966,242.6950,286.383655,444.280359,237.820321,240.814765,316.680463,-2.415128,258.923003,260.795585,227.340505,252.767556
2022-10-10 04:00:00+00:00,616.1836,237.8625,279.348804,435.605443,229.183932,230.356701,306.816358,-2.422937,252.514878,254.479060,221.266393,245.137111


timestamp
2020-02-25 05:00:00+00:00   -1.0
2020-02-26 05:00:00+00:00   -1.0
Name: signal, dtype: float64

In [10]:
cum_returns = {}
for i in range(1,100):
    
    training_begin = X.index.min()
    training_end = X.index.min() + DateOffset(weeks=i)
    
    # Slice the 'X' dataframe and 'y' Series into congruous training datasets.
    X_train = X.loc[training_begin:training_end]
    y_train = y.loc[training_begin:training_end]

# Slice the testing 'X' and 'y' datasets, starting from the end of the training data until the most recent index. 
    X_test = X.loc[training_end:]
    y_test = y.loc[training_end:]

# Use StandardScaler to scale the X_train and X_test data.
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    lrm=LogisticRegression()
    lrm.fit(X_train_scaled, y_train)
    testing_predictions = lrm.predict(X_test_scaled)
    
    lr_predictions_df = pd.DataFrame(index=X_test.index)
    lr_predictions_df['predicted_returns'] = testing_predictions

# Add in actual returns and calculate trading returns
    lr_predictions_df['actual_returns'] = df['actual_returns']
    lr_predictions_df['trading_algorithm_returns'] = lr_predictions_df['actual_returns'] * lr_predictions_df['predicted_returns']
    lr_predictions_df['actual_cumulative'] = (1 + lr_predictions_df['actual_returns']).cumprod()
    lr_predictions_df['algo_cumulative'] = (1 + lr_predictions_df['trading_algorithm_returns']).cumprod()

    final_result_actual = lr_predictions_df['actual_cumulative'][df.index[-1]]
    final_result_algo = lr_predictions_df['algo_cumulative'][df.index[-1]]
        
        
    if final_result_algo > final_result_actual:
        cum_returns[i] = final_result_algo

display(cum_returns)
max_value = max(cum_returns, key=cum_returns.get)
print(max_value)

{1: 1.3846098769357311,
 7: 0.8842254232799276,
 8: 0.5624805457004167,
 9: 0.5156243909814916,
 10: 1.21875031833489,
 11: 0.9197976096691778,
 12: 2.582081982637189,
 13: 0.709652977792044,
 14: 0.3078369820575633,
 15: 0.6566949443087959,
 16: 0.7049373206358781,
 17: 0.646160203166773,
 18: 0.612085444923454,
 22: 0.18122896912176345,
 23: 0.2072014611946847,
 24: 0.20677690952978886,
 25: 0.1490459076639944,
 26: 0.10955067075655617,
 27: 1.0624053370483784,
 28: 0.6288403885782593,
 29: 0.5153296689198171,
 30: 0.6750333277654698,
 31: 0.5984841298544875,
 38: 0.8116211264728342,
 42: 0.38221490219122206,
 46: 0.26610957764989723,
 47: 0.2698851725388304,
 48: 0.25810678692734473,
 55: 0.40157348413875504,
 57: 0.430996471216738,
 58: 0.3467293787188077,
 59: 0.38187562844510275,
 60: 0.34182729407632556,
 62: 0.3939408175790251,
 81: 0.3214056579071222,
 82: 0.3269661904253298,
 88: 0.23310887396836075,
 89: 0.270429030659164,
 90: 0.2134480147730237,
 91: 0.28695304074441785,
 

12


In [11]:
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(weeks=max_value)
    

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]


X_test = X.loc[training_end:]
y_test = y.loc[training_end:]


scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
    
lrm=LogisticRegression()
lrm.fit(X_train_scaled, y_train)
testing_predictions = lrm.predict(X_test_scaled)